# Numba func - C Callback function

This tutorial presents, how to set up a simple pipeline, that rotate images using numba_func operator.
This operator is a way for invoking user defined python code using DALI pipeline for allocating resources.

Operator is depending on Numba support https://numba.pydata.org/ which translate python and numpy code into LLVM code
which allows to reduce python interpreter overhead or completely eliminate it.

## Step-by-Step Guide
1. Let's start by importing DALI and a handful of utils.

In [11]:
from numba import cfunc, types, carray

from nvidia.dali import pipeline_def
import nvidia.dali as dali
import nvidia.dali.fn as fn
import nvidia.dali.types as dali_types
import nvidia.dali.numba_func as dali_numba

image_dir = "../data/images"
max_batch_size = 8

2. Next, we need to implement setup function for our operator. In setup function you are allowed to set output shapes basing on input shapes as well as set output data type.

In [13]:
@cfunc(dali_numba.setup_fn_sig(types.int64, types.int64), nopython=True)
def rot_image_setup(out_shape_ptr, out1_ndim, out_dtype, in_shape_ptr, in1_ndim, in_dtype, num_samples):
    in_arr = carray(in_shape_ptr, num_samples * out1_ndim)
    out_arr = carray(out_shape_ptr, num_samples * in1_ndim)
    out_type = carray(out_dtype, 1)
    out_type[0] = in_dtype
    for i in range(0, num_samples * out1_ndim, 3):
        out_arr[i] = in_arr[i + 1]
        out_arr[i + 1] = in_arr[i]
        out_arr[i + 2] = in_arr[i + 2]

To learn more about cfunc you should go straight to [numba documentation](https://numba.pydata.org/numba-doc/latest/user/cfunc.html).

`setup_fn_sig` will provide a function signature. As an arguments it takes respectively outputs and inputs types.